# Data Pre-processing steps

In [81]:
# _importing required libraries
import pandas as pd
import os
import glob

In [82]:
# _giving column names considered columns from dataset are
# timestamp (s)
# activityID  
# heart rate (bpm)
# (IMU hand) 3D-acceleration data (ms-2), scale: ±16g, resolution: 13-bit 
# (IMU hand) 3D-gyroscope data (rad/s)
subject_id = ['subject101','subject102']
col_names = ['timestamp (s)', 'activityID', 'heart rate (bpm)', 'X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2']
required_cols = [0,1,2,4,5,6,10,11,12]
main_df=pd.DataFrame()
for idx in subject_id:
    input_file_path = os.getcwd() + f'/../data/input_dat/{idx}.dat'
    df = pd.read_csv(input_file_path, header=None, names=col_names, sep='\s+', usecols=required_cols, engine='python')
    main_df=main_df.append(df)

In [83]:
main_df.reset_index()

,index,timestamp (s),activityID,heart rate (bpm),X1,Y1,Z1,X2,Y2,Z2
0,0,8.38,0,104.0,2.372230,8.600740,3.51048,-0.092217,0.056812,-0.015845
1,1,8.39,0,NaN,2.188370,8.565600,3.66179,-0.024413,0.047758,0.006474
2,2,8.40,0,NaN,2.373570,8.601070,3.54898,-0.057976,0.032574,-0.006988
3,3,8.41,0,NaN,2.074730,8.528530,3.66021,-0.002352,0.032810,-0.003747
4,4,8.42,0,NaN,2.229360,8.831220,3.70000,0.012269,0.018305,-0.053325
...,...,...,...,...,...,...,...,...,...,...
823412,446995,4475.59,0,NaN,-0.040828,-0.050765,9.77696,-0.028116,0.001145,0.016620
823413,446996,4475.60,0,NaN,-0.004109,-0.013453,9.73890,0.010495,0.006091,-0.040373
823414,446997,4475.61,0,NaN,-0.073003,-0.011192,9.93029,-0.005155,0.028014,-0.023993
823415,446998,4475.62,0,NaN,0.189650,0.099912,9.85688,-0.012021,0.011733,-0.003718


In [84]:
# _showing null values count for each column
main_df.isna().sum()

timestamp (s)            0
activityID               0
heart rate (bpm)    748233
X1                    4183
Y1                    4183
Z1                    4183
X2                    4183
Y2                    4183
Z2                    4183
dtype: int64

In [85]:
# _dropping heart rate column
del main_df['heart rate (bpm)']
# _dropping rows which has all NaN values in all sensor fields
main_df.dropna(how='all', subset= ['X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2'], inplace= True)
main_df[pd.isnull(main_df).any(axis=1)]

,timestamp (s),activityID,X1,Y1,Z1,X2,Y2,Z2


In [86]:
# _showing null values count for each column
main_df.isna().sum()

timestamp (s)    0
activityID       0
X1               0
Y1               0
Z1               0
X2               0
Y2               0
Z2               0
dtype: int64

In [87]:
# _activityID 0 represents trasient activities which are not useful in this project.
# _droping the rows whose activityID column is 0
# _considering only three activities for basic testing
main_df = main_df[main_df['activityID'].isin([1,4])] 
main_df.groupby(['activityID']).agg(['count'])

,timestamp (s),X1,Y1,Z1,X2,Y2,Z2
,count,count,count,count,count,count,count
activityID,,,,,,,
1,50613,50613,50613,50613,50613,50613,50613
4,53344,53344,53344,53344,53344,53344,53344


In [88]:
main_df


,timestamp (s),activityID,X1,Y1,Z1,X2,Y2,Z2
2928,37.66,1,2.21530,8.27915,5.587530,-0.004750,0.037579,-0.011145
2929,37.67,1,2.29196,7.67288,5.744670,-0.171710,0.025479,-0.009538
2930,37.68,1,2.29090,7.14240,5.823420,-0.238241,0.011214,0.000831
2931,37.69,1,2.21800,7.14365,5.899300,-0.192912,0.019053,0.013374
2932,37.70,1,2.30106,7.25857,6.092590,-0.069961,-0.018328,0.004582
...,...,...,...,...,...,...,...,...
310036,3106.00,4,-9.15511,4.49408,0.888468,-1.304900,0.559311,-0.541888
310037,3106.01,4,-8.93247,4.72831,1.814710,-1.021440,0.532620,-0.448802
310038,3106.02,4,-9.31312,4.58882,2.848640,-0.594921,0.470129,-0.260582
310039,3106.03,4,-9.98799,4.97413,2.914200,-0.104927,0.394412,-0.076974


In [89]:
window_length = 30
window_overlap = 10
max_window_index = len(main_df)
a1=main_df[['X1', 'Y1','Z1']]
window_index = 0
Features=[]
while window_index <= (max_window_index - window_length):
    a=a1.iloc[window_index:window_index+window_length]
    c=a1.values.tolist()
    print((window_index+window_length)-1)
    features=[(c[(window_index+window_length)-1][1]-c[window_index][1])-(c[(window_index+window_length)-1][0]-c[window_index][0]),(c[(window_index+window_length)-1][2]-c[window_index][2])-(c[(window_index+window_length)-1][0]-c[window_index][0])]
    Features.append(features)
    window_index += window_overlap



29
39
49
59
69
79
89
99
109
119
129
139
149
159
169
179
189
199
209
219
229
239
249
259
269
279
289
299
309
319
329
339
349
359
369
379
389
399
409
419
429
439
449
459
469
479
489
499
509
519
529
539
549
559
569
579
589
599
609
619
629
639
649
659
669
679
689
699
709
719
729
739
749
759
769
779
789
799
809
819
829
839
849
859
869
879
889
899
909
919
929
939
949
959
969
979
989
999
1009
1019
1029
1039
1049
1059
1069
1079
1089
1099
1109
1119
1129
1139
1149
1159
1169
1179
1189
1199
1209
1219
1229
1239
1249
1259
1269
1279
1289
1299
1309
1319
1329
1339
1349
1359
1369
1379
1389
1399
1409
1419
1429
1439
1449
1459
1469
1479
1489
1499
1509
1519
1529
1539
1549
1559
1569
1579
1589
1599
1609
1619
1629
1639
1649
1659
1669
1679
1689
1699
1709
1719
1729
1739
1749
1759
1769
1779
1789
1799
1809
1819
1829
1839
1849
1859
1869
1879
1889
1899
1909
1919
1929
1939
1949
1959
1969
1979
1989
1999
2009
2019
2029
2039
2049
2059
2069
2079
2089
2099
2109
2119
2129
2139
2149
2159
2169
2179
2189
2199
2209
2219
2229
2

15539
15549
15559
15569
15579
15589
15599
15609
15619
15629
15639
15649
15659
15669
15679
15689
15699
15709
15719
15729
15739
15749
15759
15769
15779
15789
15799
15809
15819
15829
15839
15849
15859
15869
15879
15889
15899
15909
15919
15929
15939
15949
15959
15969
15979
15989
15999
16009
16019
16029
16039
16049
16059
16069
16079
16089
16099
16109
16119
16129
16139
16149
16159
16169
16179
16189
16199
16209
16219
16229
16239
16249
16259
16269
16279
16289
16299
16309
16319
16329
16339
16349
16359
16369
16379
16389
16399
16409
16419
16429
16439
16449
16459
16469
16479
16489
16499
16509
16519
16529
16539
16549
16559
16569
16579
16589
16599
16609
16619
16629
16639
16649
16659
16669
16679
16689
16699
16709
16719
16729
16739
16749
16759
16769
16779
16789
16799
16809
16819
16829
16839
16849
16859
16869
16879
16889
16899
16909
16919
16929
16939
16949
16959
16969
16979
16989
16999
17009
17019
17029
17039
17049
17059
17069
17079
17089
17099
17109
17119
17129
17139
17149
17159
17169
17179
17189
1719

29209
29219
29229
29239
29249
29259
29269
29279
29289
29299
29309
29319
29329
29339
29349
29359
29369
29379
29389
29399
29409
29419
29429
29439
29449
29459
29469
29479
29489
29499
29509
29519
29529
29539
29549
29559
29569
29579
29589
29599
29609
29619
29629
29639
29649
29659
29669
29679
29689
29699
29709
29719
29729
29739
29749
29759
29769
29779
29789
29799
29809
29819
29829
29839
29849
29859
29869
29879
29889
29899
29909
29919
29929
29939
29949
29959
29969
29979
29989
29999
30009
30019
30029
30039
30049
30059
30069
30079
30089
30099
30109
30119
30129
30139
30149
30159
30169
30179
30189
30199
30209
30219
30229
30239
30249
30259
30269
30279
30289
30299
30309
30319
30329
30339
30349
30359
30369
30379
30389
30399
30409
30419
30429
30439
30449
30459
30469
30479
30489
30499
30509
30519
30529
30539
30549
30559
30569
30579
30589
30599
30609
30619
30629
30639
30649
30659
30669
30679
30689
30699
30709
30719
30729
30739
30749
30759
30769
30779
30789
30799
30809
30819
30829
30839
30849
30859
3086

42869
42879
42889
42899
42909
42919
42929
42939
42949
42959
42969
42979
42989
42999
43009
43019
43029
43039
43049
43059
43069
43079
43089
43099
43109
43119
43129
43139
43149
43159
43169
43179
43189
43199
43209
43219
43229
43239
43249
43259
43269
43279
43289
43299
43309
43319
43329
43339
43349
43359
43369
43379
43389
43399
43409
43419
43429
43439
43449
43459
43469
43479
43489
43499
43509
43519
43529
43539
43549
43559
43569
43579
43589
43599
43609
43619
43629
43639
43649
43659
43669
43679
43689
43699
43709
43719
43729
43739
43749
43759
43769
43779
43789
43799
43809
43819
43829
43839
43849
43859
43869
43879
43889
43899
43909
43919
43929
43939
43949
43959
43969
43979
43989
43999
44009
44019
44029
44039
44049
44059
44069
44079
44089
44099
44109
44119
44129
44139
44149
44159
44169
44179
44189
44199
44209
44219
44229
44239
44249
44259
44269
44279
44289
44299
44309
44319
44329
44339
44349
44359
44369
44379
44389
44399
44409
44419
44429
44439
44449
44459
44469
44479
44489
44499
44509
44519
4452

56539
56549
56559
56569
56579
56589
56599
56609
56619
56629
56639
56649
56659
56669
56679
56689
56699
56709
56719
56729
56739
56749
56759
56769
56779
56789
56799
56809
56819
56829
56839
56849
56859
56869
56879
56889
56899
56909
56919
56929
56939
56949
56959
56969
56979
56989
56999
57009
57019
57029
57039
57049
57059
57069
57079
57089
57099
57109
57119
57129
57139
57149
57159
57169
57179
57189
57199
57209
57219
57229
57239
57249
57259
57269
57279
57289
57299
57309
57319
57329
57339
57349
57359
57369
57379
57389
57399
57409
57419
57429
57439
57449
57459
57469
57479
57489
57499
57509
57519
57529
57539
57549
57559
57569
57579
57589
57599
57609
57619
57629
57639
57649
57659
57669
57679
57689
57699
57709
57719
57729
57739
57749
57759
57769
57779
57789
57799
57809
57819
57829
57839
57849
57859
57869
57879
57889
57899
57909
57919
57929
57939
57949
57959
57969
57979
57989
57999
58009
58019
58029
58039
58049
58059
58069
58079
58089
58099
58109
58119
58129
58139
58149
58159
58169
58179
58189
5819

70219
70229
70239
70249
70259
70269
70279
70289
70299
70309
70319
70329
70339
70349
70359
70369
70379
70389
70399
70409
70419
70429
70439
70449
70459
70469
70479
70489
70499
70509
70519
70529
70539
70549
70559
70569
70579
70589
70599
70609
70619
70629
70639
70649
70659
70669
70679
70689
70699
70709
70719
70729
70739
70749
70759
70769
70779
70789
70799
70809
70819
70829
70839
70849
70859
70869
70879
70889
70899
70909
70919
70929
70939
70949
70959
70969
70979
70989
70999
71009
71019
71029
71039
71049
71059
71069
71079
71089
71099
71109
71119
71129
71139
71149
71159
71169
71179
71189
71199
71209
71219
71229
71239
71249
71259
71269
71279
71289
71299
71309
71319
71329
71339
71349
71359
71369
71379
71389
71399
71409
71419
71429
71439
71449
71459
71469
71479
71489
71499
71509
71519
71529
71539
71549
71559
71569
71579
71589
71599
71609
71619
71629
71639
71649
71659
71669
71679
71689
71699
71709
71719
71729
71739
71749
71759
71769
71779
71789
71799
71809
71819
71829
71839
71849
71859
71869
7187

83899
83909
83919
83929
83939
83949
83959
83969
83979
83989
83999
84009
84019
84029
84039
84049
84059
84069
84079
84089
84099
84109
84119
84129
84139
84149
84159
84169
84179
84189
84199
84209
84219
84229
84239
84249
84259
84269
84279
84289
84299
84309
84319
84329
84339
84349
84359
84369
84379
84389
84399
84409
84419
84429
84439
84449
84459
84469
84479
84489
84499
84509
84519
84529
84539
84549
84559
84569
84579
84589
84599
84609
84619
84629
84639
84649
84659
84669
84679
84689
84699
84709
84719
84729
84739
84749
84759
84769
84779
84789
84799
84809
84819
84829
84839
84849
84859
84869
84879
84889
84899
84909
84919
84929
84939
84949
84959
84969
84979
84989
84999
85009
85019
85029
85039
85049
85059
85069
85079
85089
85099
85109
85119
85129
85139
85149
85159
85169
85179
85189
85199
85209
85219
85229
85239
85249
85259
85269
85279
85289
85299
85309
85319
85329
85339
85349
85359
85369
85379
85389
85399
85409
85419
85429
85439
85449
85459
85469
85479
85489
85499
85509
85519
85529
85539
85549
8555

97569
97579
97589
97599
97609
97619
97629
97639
97649
97659
97669
97679
97689
97699
97709
97719
97729
97739
97749
97759
97769
97779
97789
97799
97809
97819
97829
97839
97849
97859
97869
97879
97889
97899
97909
97919
97929
97939
97949
97959
97969
97979
97989
97999
98009
98019
98029
98039
98049
98059
98069
98079
98089
98099
98109
98119
98129
98139
98149
98159
98169
98179
98189
98199
98209
98219
98229
98239
98249
98259
98269
98279
98289
98299
98309
98319
98329
98339
98349
98359
98369
98379
98389
98399
98409
98419
98429
98439
98449
98459
98469
98479
98489
98499
98509
98519
98529
98539
98549
98559
98569
98579
98589
98599
98609
98619
98629
98639
98649
98659
98669
98679
98689
98699
98709
98719
98729
98739
98749
98759
98769
98779
98789
98799
98809
98819
98829
98839
98849
98859
98869
98879
98889
98899
98909
98919
98929
98939
98949
98959
98969
98979
98989
98999
99009
99019
99029
99039
99049
99059
99069
99079
99089
99099
99109
99119
99129
99139
99149
99159
99169
99179
99189
99199
99209
99219
9922

In [90]:
len(Features)

10393

In [91]:
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters = 20,random_state=1234)
predict_clusters=kmeans_model.fit_predict(Features)

In [92]:
data=list(zip(Features,predict_clusters))
clustered_df=pd.DataFrame(data,columns=['slope','predicted_clusters'])

In [93]:
clustered_df

,slope,predicted_clusters
0,"[-0.7558200000000004, 0.30693999999999955]",5
1,"[-0.6102300000000014, -0.03359000000000023]",5
2,"[0.56955, -2.46618]",5
3,"[-0.2589700000000006, -0.5955300000000001]",5
4,"[-1.5899900000000002, -1.8854899999999999]",18
...,...,...
10388,"[1.0579500000000008, 0.6899920000000006]",5
10389,"[-4.97746, -0.24979200000000013]",11
10390,"[5.610429999999999, 5.350925999999999]",0
10391,"[1.843490000000001, 0.8667270000000002]",5


In [94]:
centroids=kmeans_model.cluster_centers_

In [119]:
input_file_path1 = os.getcwd() + f'/../data/input_dat/subject102.dat'
df1 = pd.read_csv(input_file_path1, header=None, names=col_names, sep='\s+', usecols=required_cols, engine='python')
   

In [120]:
df1

,timestamp (s),activityID,heart rate (bpm),X1,Y1,Z1,X2,Y2,Z2
0,5.64,0,NaN,2.791430,7.553890,-7.06374,1.016400,-0.289410,1.382070
1,5.65,0,NaN,2.860860,7.438140,-7.21626,1.082690,-0.393965,1.609350
2,5.66,0,100.0,2.809970,7.094300,-7.52376,1.148290,-0.521087,1.807480
3,5.67,0,NaN,2.771380,6.980770,-7.52396,1.248880,-0.511985,1.950310
4,5.68,0,NaN,2.767100,7.131190,-7.67848,1.271780,-0.525704,2.013670
...,...,...,...,...,...,...,...,...,...
446995,4475.59,0,NaN,-0.040828,-0.050765,9.77696,-0.028116,0.001145,0.016620
446996,4475.60,0,NaN,-0.004109,-0.013453,9.73890,0.010495,0.006091,-0.040373
446997,4475.61,0,NaN,-0.073003,-0.011192,9.93029,-0.005155,0.028014,-0.023993
446998,4475.62,0,NaN,0.189650,0.099912,9.85688,-0.012021,0.011733,-0.003718


In [121]:
df1.isna().sum()

timestamp (s)            0
activityID               0
heart rate (bpm)    406205
X1                    2729
Y1                    2729
Z1                    2729
X2                    2729
Y2                    2729
Z2                    2729
dtype: int64

In [122]:
del df1['heart rate (bpm)']
# _dropping rows which has all NaN values in all sensor fields
df1.dropna(how='all', subset= ['X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2'], inplace= True)
df1[pd.isnull(df1).any(axis=1)]

,timestamp (s),activityID,X1,Y1,Z1,X2,Y2,Z2


In [123]:
df_new = df1[df1['activityID'].isin([1])] 

In [124]:
df_new

,timestamp (s),activityID,X1,Y1,Z1,X2,Y2,Z2
4956,55.20,1,-6.88206,6.54265,2.83997,-0.003550,-0.046823,-0.009676
4957,55.21,1,-6.84775,6.61723,2.72477,-0.012985,-0.025277,-0.010775
4958,55.22,1,-6.96887,6.46467,2.53104,-0.013640,-0.017291,-0.023895
4959,55.23,1,-6.85123,6.50274,2.64813,0.025889,0.010136,-0.025000
4960,55.24,1,-6.80836,6.46585,2.80285,0.039428,-0.022150,-0.051769
...,...,...,...,...,...,...,...,...
28381,289.45,1,-8.68877,2.71381,1.63272,0.277572,-0.117667,0.030982
28382,289.46,1,-8.61051,2.71390,1.71087,0.270114,-0.133827,-0.033978
28383,289.47,1,-8.82976,2.64178,1.93875,0.255291,-0.138561,-0.074433
28384,289.48,1,-8.94848,2.64132,1.78302,0.283784,-0.153951,-0.074558


In [125]:
window_length = 30
window_overlap = 10
max_window_index = len(df_new)
a1=df_new[['X1', 'Y1','Z1']]
window_index = 0
Features=[]
while window_index <= (max_window_index - window_length):
    a=a1.iloc[window_index:window_index+window_length]
    c=a1.values.tolist()
    print((window_index+window_length)-1)
    features=[(c[(window_index+window_length)-1][1]-c[window_index][1])-(c[(window_index+window_length)-1][0]-c[window_index][0]),(c[(window_index+window_length)-1][2]-c[window_index][2])-(c[(window_index+window_length)-1][0]-c[window_index][0])]
    Features.append(features)
    window_index += window_overlap

29
39
49
59
69
79
89
99
109
119
129
139
149
159
169
179
189
199
209
219
229
239
249
259
269
279
289
299
309
319
329
339
349
359
369
379
389
399
409
419
429
439
449
459
469
479
489
499
509
519
529
539
549
559
569
579
589
599
609
619
629
639
649
659
669
679
689
699
709
719
729
739
749
759
769
779
789
799
809
819
829
839
849
859
869
879
889
899
909
919
929
939
949
959
969
979
989
999
1009
1019
1029
1039
1049
1059
1069
1079
1089
1099
1109
1119
1129
1139
1149
1159
1169
1179
1189
1199
1209
1219
1229
1239
1249
1259
1269
1279
1289
1299
1309
1319
1329
1339
1349
1359
1369
1379
1389
1399
1409
1419
1429
1439
1449
1459
1469
1479
1489
1499
1509
1519
1529
1539
1549
1559
1569
1579
1589
1599
1609
1619
1629
1639
1649
1659
1669
1679
1689
1699
1709
1719
1729
1739
1749
1759
1769
1779
1789
1799
1809
1819
1829
1839
1849
1859
1869
1879
1889
1899
1909
1919
1929
1939
1949
1959
1969
1979
1989
1999
2009
2019
2029
2039
2049
2059
2069
2079
2089
2099
2109
2119
2129
2139
2149
2159
2169
2179
2189
2199
2209
2219
2229
2

15589
15599
15609
15619
15629
15639
15649
15659
15669
15679
15689
15699
15709
15719
15729
15739
15749
15759
15769
15779
15789
15799
15809
15819
15829
15839
15849
15859
15869
15879
15889
15899
15909
15919
15929
15939
15949
15959
15969
15979
15989
15999
16009
16019
16029
16039
16049
16059
16069
16079
16089
16099
16109
16119
16129
16139
16149
16159
16169
16179
16189
16199
16209
16219
16229
16239
16249
16259
16269
16279
16289
16299
16309
16319
16329
16339
16349
16359
16369
16379
16389
16399
16409
16419
16429
16439
16449
16459
16469
16479
16489
16499
16509
16519
16529
16539
16549
16559
16569
16579
16589
16599
16609
16619
16629
16639
16649
16659
16669
16679
16689
16699
16709
16719
16729
16739
16749
16759
16769
16779
16789
16799
16809
16819
16829
16839
16849
16859
16869
16879
16889
16899
16909
16919
16929
16939
16949
16959
16969
16979
16989
16999
17009
17019
17029
17039
17049
17059
17069
17079
17089
17099
17109
17119
17129
17139
17149
17159
17169
17179
17189
17199
17209
17219
17229
17239
1724

In [127]:
len(Features)

2340

In [128]:
centroids=kmeans_model.cluster_centers_

In [152]:
test_predict=kmeans_model.predict(Features)

In [153]:
test_predict

array([5, 5, 5, ..., 5, 5, 5], dtype=int32)

In [145]:
df_new1 = df1[df1['activityID'].isin([6])] 
df_new1

,timestamp (s),activityID,X1,Y1,Z1,X2,Y2,Z2
358225,3587.89,6,-4.58387,4.06261,7.53974,-0.043398,0.049529,-0.038003
358226,3587.90,6,-4.70017,4.02488,7.46115,-0.093222,0.074255,-0.019473
358227,3587.91,6,-4.69964,3.91140,7.50003,-0.089209,0.085999,-0.020712
358228,3587.92,6,-4.62593,3.83460,7.46290,-0.124500,0.051974,-0.018929
358229,3587.93,6,-4.88993,4.10188,7.41956,-0.109390,0.075054,-0.040548
...,...,...,...,...,...,...,...,...
383328,3838.92,6,-1.27957,6.90992,6.38652,-0.056944,-0.029252,0.024453
383329,3838.93,6,-1.27717,6.87265,6.46366,-0.057943,0.031090,0.029731
383330,3838.94,6,-1.19865,6.91067,6.54168,-0.085186,0.003280,0.039340
383331,3838.95,6,-1.04668,6.83396,6.58270,-0.094369,0.037039,0.056186


In [146]:
window_length = 30
window_overlap = 10
max_window_index = len(df_new1)
a1=df_new1[['X1', 'Y1','Z1']]
window_index = 0
Features2=[]
while window_index <= (max_window_index - window_length):
    a=a1.iloc[window_index:window_index+window_length]
    c=a1.values.tolist()
    print((window_index+window_length)-1)
    features=[(c[(window_index+window_length)-1][1]-c[window_index][1])-(c[(window_index+window_length)-1][0]-c[window_index][0]),(c[(window_index+window_length)-1][2]-c[window_index][2])-(c[(window_index+window_length)-1][0]-c[window_index][0])]
    Features2.append(features)
    window_index += window_overlap

29
39
49
59
69
79
89
99
109
119
129
139
149
159
169
179
189
199
209
219
229
239
249
259
269
279
289
299
309
319
329
339
349
359
369
379
389
399
409
419
429
439
449
459
469
479
489
499
509
519
529
539
549
559
569
579
589
599
609
619
629
639
649
659
669
679
689
699
709
719
729
739
749
759
769
779
789
799
809
819
829
839
849
859
869
879
889
899
909
919
929
939
949
959
969
979
989
999
1009
1019
1029
1039
1049
1059
1069
1079
1089
1099
1109
1119
1129
1139
1149
1159
1169
1179
1189
1199
1209
1219
1229
1239
1249
1259
1269
1279
1289
1299
1309
1319
1329
1339
1349
1359
1369
1379
1389
1399
1409
1419
1429
1439
1449
1459
1469
1479
1489
1499
1509
1519
1529
1539
1549
1559
1569
1579
1589
1599
1609
1619
1629
1639
1649
1659
1669
1679
1689
1699
1709
1719
1729
1739
1749
1759
1769
1779
1789
1799
1809
1819
1829
1839
1849
1859
1869
1879
1889
1899
1909
1919
1929
1939
1949
1959
1969
1979
1989
1999
2009
2019
2029
2039
2049
2059
2069
2079
2089
2099
2109
2119
2129
2139
2149
2159
2169
2179
2189
2199
2209
2219
2229
2

15559
15569
15579
15589
15599
15609
15619
15629
15639
15649
15659
15669
15679
15689
15699
15709
15719
15729
15739
15749
15759
15769
15779
15789
15799
15809
15819
15829
15839
15849
15859
15869
15879
15889
15899
15909
15919
15929
15939
15949
15959
15969
15979
15989
15999
16009
16019
16029
16039
16049
16059
16069
16079
16089
16099
16109
16119
16129
16139
16149
16159
16169
16179
16189
16199
16209
16219
16229
16239
16249
16259
16269
16279
16289
16299
16309
16319
16329
16339
16349
16359
16369
16379
16389
16399
16409
16419
16429
16439
16449
16459
16469
16479
16489
16499
16509
16519
16529
16539
16549
16559
16569
16579
16589
16599
16609
16619
16629
16639
16649
16659
16669
16679
16689
16699
16709
16719
16729
16739
16749
16759
16769
16779
16789
16799
16809
16819
16829
16839
16849
16859
16869
16879
16889
16899
16909
16919
16929
16939
16949
16959
16969
16979
16989
16999
17009
17019
17029
17039
17049
17059
17069
17079
17089
17099
17109
17119
17129
17139
17149
17159
17169
17179
17189
17199
17209
1721

In [147]:
test_predict1=kmeans_model.predict(Features2)

In [148]:
test_predict1=test_predict1[0:2340]

In [150]:
len(test_predict)

2340

In [151]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([test_predict], [test_predict1])



array([[0.80905086]])